In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
PATH =  "ashrae-energy-prediction/"

# Copy Kernel 
# URL : "https://www.kaggle.com/isaienkov/lightgbm-fe-1-19"

In [2]:
import kaggle

# what's this function??

In [4]:
def degToCompass(num):
    val=int((num/22.5)+.5)
    arr=[i for i in range(0,16)]
    return arr[(val % 16)]

新しいオリジナル関数…良かったらkaggle.py に追加しよう。

"train.pkl" was created by merging building_metadata.csv,weather_train.csv,train.csv

In [5]:
train_df = pd.read_pickle(PATH + "train.pkl")

In [6]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,year
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016


In [ ]:
train_df["timestamp"] = pd.to_datetime(train_df.timestamp)
train_df["weekday"] = train_df.timestamp.dt.weekday # new variable weekday
train_df["hour"] = train_df.timestamp.dt.hour # new variable hour
train_df["year_built"] = train_df.year_built - 1900 # adjusting scale by substract 1900 !
train_df['square_feet'] = np.log(train_df['square_feet']) # adjusting scale by log

In [9]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,year,weekday,hour
0,0,0,2016-01-01,0.0,0,Education,8.913550,108.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0
1,1,0,2016-01-01,0.0,0,Education,7.908387,104.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0
2,2,0,2016-01-01,0.0,0,Education,8.589700,91.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0
3,3,0,2016-01-01,0.0,0,Education,10.072597,102.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0
4,4,0,2016-01-01,0.0,0,Education,11.666565,75.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0


# ??

In [11]:
def average_imputation(df, column_name):
    imputation = df.groupby(['timestamp'])[column_name].mean()
    
    df.loc[df[column_name].isnull(), column_name] = df[df[column_name].isnull()][[column_name]].apply(lambda x: imputation[df['timestamp'][x.index]].values)
    del imputation
    return df

In [ ]:
train_df = average_imputation(train_df, 'wind_speed')
train_df = average_imputation(train_df, 'wind_direction')

# this value means … ? 
beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]
for item in beaufort:
    train_df.loc[(train_df['wind_speed']>=item[1]) & (train_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

In [15]:
del train_df["timestamp"] # delete value

In [16]:
train_df.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,year,weekday,hour,beaufort_scale
0,0,0,0.0,0,Education,8.913550,108.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0,0.0
1,1,0,0.0,0,Education,7.908387,104.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0,0.0
2,2,0,0.0,0,Education,8.589700,91.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0,0.0
3,3,0,0.0,0,Education,10.072597,102.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0,0.0
4,4,0,0.0,0,Education,11.666565,75.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,2016,4,0,0.0


In [17]:
train_df['wind_direction'] = train_df['wind_direction'].apply(degToCompass) # scale by custom function
train_df['beaufort_scale'] = train_df['beaufort_scale'].astype(np.uint8) # 符号あり8ビット整数型
train_df["wind_direction"] = train_df['wind_direction'].astype(np.uint8) # 符号あり8ビット整数型
train_df["meter"] = train_df['meter'].astype(np.uint8) # 符号あり8ビット整数型
train_df["site_id"] = train_df['site_id'].astype(np.uint8) # 符号あり8ビット整数型

# LabelEncoding 

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df["primary_use"] = le.fit_transform(train_df["primary_use"])

categoricals = ["site_id", "building_id", "primary_use", "hour", "weekday", "meter",  "wind_direction"]

# lgbm make branch by comparing value.so , TE is better than LE??

In [22]:
train_df.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,year,weekday,hour,beaufort_scale
0,0,0,0.0,0,0,8.913550,108.0,NaN,25.0,6.0,20.0,NaN,1019.7,0,0.0,1,2016,4,0,0
1,1,0,0.0,0,0,7.908387,104.0,NaN,25.0,6.0,20.0,NaN,1019.7,0,0.0,1,2016,4,0,0
2,2,0,0.0,0,0,8.589700,91.0,NaN,25.0,6.0,20.0,NaN,1019.7,0,0.0,1,2016,4,0,0
3,3,0,0.0,0,0,10.072597,102.0,NaN,25.0,6.0,20.0,NaN,1019.7,0,0.0,1,2016,4,0,0
4,4,0,0.0,0,0,11.666565,75.0,NaN,25.0,6.0,20.0,NaN,1019.7,0,0.0,1,2016,4,0,0


In [20]:
drop_cols = ["sea_level_pressure", "wind_speed"]

numericals = ["square_feet", "year_built", "air_temperature", "cloud_coverage",
              "dew_temperature", 'precip_depth_1_hr', 'floor_count', 'beaufort_scale']

feat_cols = categoricals + numericals

In [21]:
feat_cols

['site_id',
 'building_id',
 'primary_use',
 'hour',
 'weekday',
 'meter',
 'wind_direction',
 'square_feet',
 'year_built',
 'air_temperature',
 'cloud_coverage',
 'dew_temperature',
 'precip_depth_1_hr',
 'floor_count',
 'beaufort_scale']

In [23]:
train_target = np.log1p(train_df["meter_reading"]) # scaling (log(1 +y)) because this evaluation function in this competition is RMSLE
del train_df["meter_reading"] 

train_df = train_df.drop(drop_cols, axis = 1)

In [24]:
train_df.head()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,wind_direction,month,year,weekday,hour,beaufort_scale
0,0,0,0,0,8.913550,108.0,NaN,25.0,6.0,20.0,NaN,0,1,2016,4,0,0
1,1,0,0,0,7.908387,104.0,NaN,25.0,6.0,20.0,NaN,0,1,2016,4,0,0
2,2,0,0,0,8.589700,91.0,NaN,25.0,6.0,20.0,NaN,0,1,2016,4,0,0
3,3,0,0,0,10.072597,102.0,NaN,25.0,6.0,20.0,NaN,0,1,2016,4,0,0
4,4,0,0,0,11.666565,75.0,NaN,25.0,6.0,20.0,NaN,0,1,2016,4,0,0


# for now, my original feature year and month drop.

In [25]:
train_df.drop(["year","month"],axis = 1,inplace = True)

# What is tqdm ??

In [26]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

In [27]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.25,
            'subsample_freq': 1,
            'learning_rate': 0.4,
            'num_leaves': 20,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            }

In [28]:

folds = 4
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

# 同じビルの比率にしない　
# →　評価データにあるビルIDが訓練データにないという事態が生じる可能性がある
# →　評価データと訓練データをビルIDの存在比率が等しくなるように分ける必要がある。

In [30]:
models = []
for train_index, val_index in kf.split(train_df, train_df['building_id']):
    train_X = train_df[feat_cols].iloc[train_index]
    val_X = train_df[feat_cols].iloc[val_index]
    
    train_y = train_target.iloc[train_index]
    val_y = train_target.iloc[val_index]
    
    lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
    lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)
    
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    
    models.append(gbm)

/home/yada_yuki_entertainment/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.03501	valid_1's rmse: 1.03895
[200]	training's rmse: 1.00723	valid_1's rmse: 1.01409
[300]	training's rmse: 0.992541	valid_1's rmse: 1.0017
[400]	training's rmse: 0.979625	valid_1's rmse: 0.991879
[500]	training's rmse: 0.968748	valid_1's rmse: 0.982977
Did not meet early stopping. Best iteration is:
[500]	training's rmse: 0.968748	valid_1's rmse: 0.982977
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.03803	valid_1's rmse: 1.04459
[200]	training's rmse: 1.01048	valid_1's rmse: 1.0202
[300]	training's rmse: 0.996307	valid_1's rmse: 1.00885
[400]	training's rmse: 0.985529	valid_1's rmse: 1.0005
[500]	training's rmse: 0.976272	valid_1's rmse: 0.993165
Did not meet early stopping. Best iteration is:
[500]	training's rmse: 0.976272	valid_1's rmse: 0.993165
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.04738	valid_1's rmse: 1

# delete df which would not use 

In [32]:
import gc
del train_df, train_X, val_X, lgb_train, lgb_eval, train_y, val_y, train_target
gc.collect()

81

# test_df

In [34]:
test_df = pd.read_csv(PATH + "test.csv")
building_df = pd.read_csv(PATH + "building_metadata.csv")
test_df = test_df.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
del building_df
gc.collect()
test_df["primary_use"] = le.transform(test_df["primary_use"])

weather_test = pd.read_csv(PATH + "weather_test.csv")

test_df = test_df.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
del weather_test
gc.collect()

28

# FE same as train

In [35]:
test_df["timestamp"] = pd.to_datetime(test_df["timestamp"])
test_df["hour"] = test_df["timestamp"].dt.hour
test_df["weekday"] = test_df["timestamp"].dt.weekday
test_df["weekday"] = test_df['weekday'].astype(np.uint8)
test_df["hour"] = test_df['hour'].astype(np.uint8)
test_df['year_built'] = test_df['year_built']-1900
test_df['square_feet'] = np.log(test_df['square_feet'])

test_df = average_imputation(test_df, 'wind_speed')
test_df = average_imputation(test_df, 'wind_direction')

for item in beaufort:
    test_df.loc[(test_df['wind_speed']>=item[1]) & (test_df['wind_speed']<item[2]), 'beaufort_scale'] = item[0]
test_df['wind_direction'] = test_df['wind_direction'].apply(degToCompass)

test_df['wind_direction'] = test_df['wind_direction'].apply(degToCompass)
test_df['beaufort_scale'] = test_df['beaufort_scale'].astype(np.uint8)
test_df["wind_direction"] = test_df['wind_direction'].astype(np.uint8)
test_df["meter"] = test_df['meter'].astype(np.uint8)
test_df["site_id"] = test_df['site_id'].astype(np.uint8)

test_df = test_df[feat_cols]

In [37]:
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test_df.shape[0]/50000)))):
    res.append(np.expm1(sum([model.predict(test_df.iloc[i:i+step_size]) for model in models])/folds))
    i+=step_size

100%|██████████| 834/834 [23:37<00:00,  1.68s/it]


In [38]:
y_prediction = np.concatenate(res)

In [41]:
%%javascript
var notebook_name = document.body.attributes['data-notebook-name'].value
IPython.notebook.kernel.execute(`NOTEBOOK_NAME = '${notebook_name}'`);

<IPython.core.display.Javascript object>

In [42]:
kaggle.generate_submission("row_id",test_df.index,"meter_reading",y_prediction,NOTEBOOK_NAME,add_word = "CopyKernel")

file_name =  sub_3_lgbm.ipynb_2019-10-26_03-32-42_CopyKernel.csv
   row_id  meter_reading
0       0      10.786174
1       1       9.014033
2       2       2.111600
3       3      24.722210
4       4      32.706067
